In [8]:
import tweepy
import numpy as np
import pandas as pd
import re
import datetime
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sb
import MeCab
mpl.rcParams['font.family'] = "IPAexGothic"
#scikit learn
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
#gensim
import gensim
from gensim import corpora, matutils

# Setting 80 characters in a column
pd.set_option("display.max_colwidth", 280)

## Gathering data

### Gethering from twitter

In [9]:
# Getting customer key and access token
# Deleted below 4 keys due to confidential information
consumer_key = ''
consumer_secret = ''
access_token = ''
access_secret = ''

In [10]:
# Getting api value
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)
api = tweepy.API(auth)

In [11]:
# Creating column list and searched tweet id
columns_name=["TW_NO","TW_TIME","TW_TEXT","FAV","RT"]
tw_id ='tarinaifutari'

In [12]:
# Creating DataFrame
tweet_data = []
for tweet in tweepy.Cursor(api.user_timeline,screen_name = tw_id,exclude_replies = True).items():
    tweet_data.append([tweet.id,tweet.created_at+datetime.timedelta(hours=9),tweet.text.replace('\n',''),tweet.favorite_count,tweet.retweet_count])
df = pd.DataFrame(tweet_data,columns=columns_name)

In [13]:
df.head(5)

,TW_NO,TW_TIME,TW_TEXT,FAV,RT
0,1211708062337126400,2019-12-31 02:58:09,…文面から棒読みが伝わりますが、なにせ第2章が始まったばかりなのでご勘弁ください。さよなら たりないふたりこんにちは たりないふたり文責 山里 https://t.co/T4WcRLwvOh,3063,447
1,1211707984801189896,2019-12-31 02:57:50,放送ご覧頂きありがとうございました。改めまして皆様、ご指導、ご鞭撻の程よろしくお願いします。文責 若林,3809,520
2,1211707917176426497,2019-12-31 02:57:34,放送ご覧頂きありがとうございました。お知らせです。Huluで完全版の配信がスタートしました。色んな理由でカットされた部分や舞台裏映像も入って3時間位あるそうです。ライブグッズの通販も始まったのでぜひ！文責 山里,2989,484
3,1209752987540705280,2019-12-25 17:29:23,どうかしてるよ…それ、大人の都合で絶対まだ言ってはいけないって説明されたでしょ？文責 山里,2779,426
4,1209724189575282688,2019-12-25 15:34:57,さよならたりないふたり完全版はHuluで放送直後から配信。テレビ版でカットしたところや舞台裏ドキュメント入って3時間弱です。←この情報だけゲットしました。最近、解禁前の情報を横流しできずすみません。 文責 若林,6397,1523


## Wrangling data

### Deleting the space

In [14]:
# Delete the space in twitter text
df['TW_TEXT'] = df['TW_TEXT'].apply(lambda x: re.sub('([あ-んア-ン一-龥ー])\s+((?=[あ-んア-ン一-龥ー]))',r'\1\2', x))

### Creating new column "person"

In [15]:
# Create person column to clarify who tweeted.
# Firstly, I filtered the word "文責山里" and "文責若林" following their rule
df['PERSON'] ='不明'

for i in range(df.shape[0]):
    if '文責若林' in df.loc[i,'TW_TEXT']:
        df.loc[i, 'PERSON']='若林'
    elif '文責山里' in df.loc[i,'TW_TEXT'] :
        df.loc[i, 'PERSON']='山里'

In [16]:
# Modifying the filter
# ['文責山里若林', '文責山里・若林'] → 双方
for b_word in ['文責山里若林', '文責山里・若林']:
    for i in range(df.shape[0]):
        if b_word in df.loc[i,'TW_TEXT']:
            df.loc[i,'PERSON']='双方'

In [17]:
# Checking if the above change is working or not
df[df['PERSON']=='双方']['TW_TEXT'].count()

4

In [18]:
# Checking the tweet not following the rule
df[df['PERSON']=='不明']['TW_TEXT'].count()

537

In [19]:
# Checking the tweet not following the rule
df[df['PERSON']=='不明']['TW_TEXT'].head()

5                                                                                                            さよなら？いや、こんにちは。 https://t.co/kb0drHfsr9
40         RT @thetvjp: 「たりないふたり」5年ぶり復活で“さよなら”のワケ企画演出・安島隆氏が語る山里亮太×若林正恭の10年 #たりないふたり #たりふた #ライブ #山里亮太 #若林正恭 #安島隆 @tarinaifutari @takashiajimahttps:…
82     たりないふたりファンの方に現状報告です。2人で会える日がなく山ちゃんはスッキリの見守り終わり、ぼくは日テレのレギュラー番組の前に安島さんが楽屋にやって来て「ハロウィンなんかは今どう思う？」と聞かれてはそれを伝言しに行くというスタイ… https://t.co/rPpQ3HRBVm
165                                                                                                                  チュウ、チュウ、次がラストの収録。色々と企みチュウ。文責悪鼠
167                                                                                                                              チュウ、チュウ、ロケチュウ。文責悪鼠
Name: TW_TEXT, dtype: object

In [20]:
# Checking the other word which we can filter the person by.
# ['文責悪鼠', '歌責若林', 'MASA', '文責・若林', '文責若']　→ 若林
# ['山里亮太'] → 山里

for w_word in ['文責悪鼠', '歌責若林', 'MASA', '文責・若林', '文責若']:
    for i in range(df.shape[0]):
        if df.loc[i,'PERSON']=='不明':
            if w_word in df.loc[i,'TW_TEXT']:
                df.loc[i,'PERSON']='若林'

for i in range(df.shape[0]):
    if df.loc[i,'PERSON']=='不明':
        if '山里亮太' in df.loc[i,'TW_TEXT']:
            df.loc[i,'PERSON']='山里'

In [21]:
# Checking the tweet not following the rule
df[df['PERSON']=='不明']['TW_TEXT'].count()

511

In [22]:
df=df[(df['PERSON']=='山里') | (df['PERSON']=='若林')].reset_index()

### Creating Person Label column

In [23]:
df['PERSON_LABEL']=''

for i in range(df.shape[0]):
    if df.loc[i, 'PERSON'] =='若林':
        df.loc[i, 'PERSON_LABEL'] =1
    elif df.loc[i, 'PERSON'] =='山里':
        df.loc[i, 'PERSON_LABEL'] =0

## Storing data into csv file

In [24]:
df.to_csv('tarifuta_analysis_ver2.csv', index=False)

### Reading csv file

In [25]:
df = pd.read_csv('tarifuta_analysis_ver2.csv')

In [26]:
df.head(3)

,index,TW_NO,TW_TIME,TW_TEXT,FAV,RT,PERSON,PERSON_LABEL
0,0,1211708062337126400,2019-12-31 02:58:09,…文面から棒読みが伝わりますが、なにせ第2章が始まったばかりなのでご勘弁ください。さよならたりないふたりこんにちはたりないふたり文責山里 https://t.co/T4WcRLwvOh,3063,447,山里,0
1,1,1211707984801189896,2019-12-31 02:57:50,放送ご覧頂きありがとうございました。改めまして皆様、ご指導、ご鞭撻の程よろしくお願いします。文責若林,3809,520,若林,1
2,2,1211707917176426497,2019-12-31 02:57:34,放送ご覧頂きありがとうございました。お知らせです。Huluで完全版の配信がスタートしました。色んな理由でカットされた部分や舞台裏映像も入って3時間位あるそうです。ライブグッズの通販も始まったのでぜひ！文責山里,2989,484,山里,0


### Normalization for MeCab

In [27]:
# encoding: utf8
from __future__ import unicode_literals
import re
import unicodedata

def unicode_normalize(cls, s):
    pt = re.compile('([{}]+)'.format(cls))

    def norm(c):
        return unicodedata.normalize('NFKC', c) if pt.match(c) else c

    s = ''.join(norm(x) for x in re.split(pt, s))
    s = re.sub('－', '-', s)
    return s

def remove_extra_spaces(s):
    s = re.sub('[ 　]+', ' ', s)
    blocks = ''.join(('\u4E00-\u9FFF',  # CJK UNIFIED IDEOGRAPHS
                      '\u3040-\u309F',  # HIRAGANA
                      '\u30A0-\u30FF',  # KATAKANA
                      '\u3000-\u303F',  # CJK SYMBOLS AND PUNCTUATION
                      '\uFF00-\uFFEF'   # HALFWIDTH AND FULLWIDTH FORMS
                      ))
    basic_latin = '\u0000-\u007F'

    def remove_space_between(cls1, cls2, s):
        p = re.compile('([{}]) ([{}])'.format(cls1, cls2))
        while p.search(s):
            s = p.sub(r'\1\2', s)
        return s

    s = remove_space_between(blocks, blocks, s)
    s = remove_space_between(blocks, basic_latin, s)
    s = remove_space_between(basic_latin, blocks, s)
    return s

def normalize_neologd(s):
    s = s.strip()
    s = unicode_normalize('０-９Ａ-Ｚａ-ｚ｡-ﾟ', s)

    def maketrans(f, t):
        return {ord(x): ord(y) for x, y in zip(f, t)}

    s = re.sub('[˗֊‐‑‒–⁃⁻₋−]+', '-', s)  # normalize hyphens
    s = re.sub('[﹣－ｰ—―─━ー]+', 'ー', s)  # normalize choonpus
    s = re.sub('[~∼∾〜〰～]', '', s)  # remove tildes
    s = re.sub('https?://[\w/:%#\$&\?\(\)~\.=\+\-…]+', '', s)  # remove https link
    s = re.sub('<[^>]*?>', '', s)  # remove https link
    s = s.translate(
        maketrans('!"#$%&\'()*+,-./:;<=>?@[¥]^_`{|}~｡､･｢｣',
              '！”＃＄％＆’（）＊＋，－．／：；＜＝＞？＠［￥］＾＿｀｛｜｝〜。、・「」'))

    s = remove_extra_spaces(s)
    s = unicode_normalize('！”＃＄％＆’（）＊＋，－．／：；＜＞？＠［￥］＾＿｀｛｜｝〜', s)  # keep ＝,・,「,」
    s = re.sub('[’]', '\'', s)
    s = re.sub('[”]', '"', s)
    return s

### DataFrameの形態要素解析

In [32]:
def get_words(df,column):
    '''
    記事群のdictについて、形態素解析してリストにして返す
    '''
    ret = []
    for i in range(df.shape[0]):
        ret.append(get_words_main(df.loc[i,column]))
    return ret

def get_words_main(content):
    '''
    一つの記事を形態素解析して返す
    '''
    return [token for token in tokenize(content)]

def tokenize(text):
    '''
    とりあえず形態素解析して全ての品詞を取り出す感じにしてる
    '''
    node = mecab.parseToNode(text)
    while node:
        if node.feature.split(',')[0] == "名詞":
            yield node.surface.lower()
        node = node.next

### Delete "文責"

In [33]:
def delete_bunseki(df,column):
    ret =[]
    re_person = '(文責(若林|悪鼠|・若林|若|山里|謎のラッパー|たり林てる恭|逃げ林|デロリアン山里))|(歌責若林)|MASA|山里亮太'
    for i in range(df.shape[0]):
        ret.append(re.sub(re_person,'',df.loc[i,column]))
    return ret

### vectorの取得

In [34]:
def get_vector(dictionary, content):
    '''
    ある記事の特徴語カウント
    '''
    tmp = dictionary.doc2bow(get_words_main(content))
    dense = list(matutils.corpus2dense([tmp], num_terms=len(dictionary)).T[0])
    return dense

# Main

In [35]:
mecab = MeCab.Tagger ('-d /usr/local/lib/mecab/dic/mecab-ipadic-neologd')

In [36]:
df['TW_TEXT']=delete_bunseki(df,'TW_TEXT')

df['TW_TEXT_VECTOR']=""
for i in range(df.shape[0]):
    df.loc[i,'TW_TEXT_VECTOR']= normalize_neologd(df.loc[i,'TW_TEXT'])

df['TW_TEXT_VECTOR']=get_words(df,'TW_TEXT_VECTOR')

dictionary = corpora.Dictionary(df['TW_TEXT_VECTOR'])
dictionary.filter_extremes(no_below=10, no_above=0.3)
dictionary.save_as_text('tarifuta_dict.txt')

In [37]:
dictionary = corpora.Dictionary.load_from_text('tarifuta_dict.txt')

In [38]:
data_train = []
label_train = []
for i in range(df.shape[0]):
    data_train.append(get_vector(dictionary, df.loc[i,'TW_TEXT']))
    label_train.append(df.loc[i,'PERSON_LABEL'])

In [39]:
# 分類器
estimator = RandomForestClassifier()

# 学習
estimator.fit(data_train, label_train)

# 学習したデータを予測にかけてみる（ズルなので正答率高くないとおかしい）
print("==== 学習データと予測データが一緒の場合")
print(estimator.score(data_train, label_train))

# 学習データと試験データに分けてみる
data_train_s, data_test_s, label_train_s, label_test_s = train_test_split(data_train, label_train, test_size=0.5)

# 分類器をもう一度定義
estimator2 = RandomForestClassifier()

# 学習
estimator2.fit(data_train_s, label_train_s)
print("==== 学習データと予測データが違う場合")
print(estimator2.score(data_test_s, label_test_s))

# グリッドサーチやってみる
tuned_parameters = [{'n_estimators': [10, 30, 50, 70, 90, 110, 130, 150], 'max_features': ['auto', 'sqrt', 'log2', None]}]

clf = GridSearchCV(RandomForestClassifier(), tuned_parameters, cv=2, scoring='accuracy', n_jobs=-1)
clf.fit(data_train_s, label_train_s)

==== 学習データと予測データが一緒の場合
0.9054996127033308
==== 学習データと予測データが違う場合
0.781733746130031


/Users/Waka/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/Users/Waka/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


GridSearchCV(cv=2, error_score='raise-deprecating',
       estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators='warn', n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False),
       fit_params=None, iid='warn', n_jobs=-1,
       param_grid=[{'n_estimators': [10, 30, 50, 70, 90, 110, 130, 150], 'max_features': ['auto', 'sqrt', 'log2', None]}],
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='accuracy', verbose=0)

In [40]:
print("==== グリッドサーチ")
print("  ベストパラメタ")
print(clf.best_estimator_)
print("  ベストスコア")
print(clf.best_score_)

==== グリッドサーチ
  ベストパラメタ
RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='sqrt', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=110, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)
  ベストスコア
0.751937984496124
